In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 67.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch

if torch.cuda.is_available():
  import cuml
  from cuml import KMeans
  from cuml.cluster import KMeans
  import cudf

import faiss
import sys
sys.path.append('//content/drive/MyDrive/Colab Notebooks/')
from eegUtils import *

import pandas as pd

In [ ]:
processedData = np.load('/content/drive/MyDrive/NeuroResearch/Data/eegCompress/processedData/origAndPredictedSVD001_block7.npz')
data = processedData['arr_0']
predicted = processedData['arr_1']
residual = predicted - data
nChannel, nSample = data.shape

dataTensor = torch.tensor(data)
if torch.cuda.is_available():
  dataTensor = dataTensor.to('cuda')
  print("Data has been moved to GPU")
else:
  print("Data is on CPU")
print(data.shape)

residual = residual.transpose()

Data has been moved to GPU
(19, 1100367)


# FAISS KMeans Training

In [ ]:
ncentroids = 4096
niter = 300
verbose = True
d = residual.shape[1]
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose, gpu=True)
kmeans.train(residual)
centroids = kmeans.centroids
centroids.shape

(4096, 19)

# CUML Kmeans Training

In [ ]:
n_clusters = 256
kmeans = KMeans(n_clusters=n_clusters, n_init="auto", verbose=True)
kmeans.fit(residual.transpose())
centroids = kmeans.cluster_centers_
#kmeans.labels_

RuntimeError: CUDA error encountered at: file=/pyenv/versions/3.12.9/lib/python3.12/site-packages/libraft/include/raft/util/cudart_utils.hpp line=148: call='cudaMemcpyAsync(dst, src, len * sizeof(Type), cudaMemcpyDefault, stream)', Reason=cudaErrorIllegalAddress:an illegal memory access was encountered
Obtained 57 stack frames
#1 in /usr/local/lib/python3.11/dist-packages/libcuvs/lib64/libcuvs.so: raft::cuda_error::cuda_error(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&) +0xbd [0x7c13e8c64dcd]
#2 in /usr/local/lib/python3.11/dist-packages/libcuvs/lib64/libcuvs.so: void raft::copy<double>(double*, double const*, unsigned long, rmm::cuda_stream_view) +0x1a0 [0x7c13e8e09fd0]
#3 in /usr/local/lib/python3.11/dist-packages/libcuvs/lib64/libcuvs.so: void cuvs::cluster::kmeans::detail::initScalableKMeansPlusPlus<double, int>(raft::resources const&, cuvs::cluster::kmeans::params const&, std::experimental::mdspan<double const, std::experimental::extents<

# Save KMeans model

In [ ]:
path = '/content/drive/MyDrive/NeuroResearch/Data/eegCompress/processedData/centroids_001_block7.npy'
np.save(path, centroids)

# Load CUML KMeans model

In [ ]:
path = '/content/drive/MyDrive/NeuroResearch/Data/eegCompress/processedData/centroids_001_block7.npy'
centroids = np.load(path)
nCentroids,_ = centroids.shape
kmeansCuml = KMeans(n_clusters=nCentroids, init=centroids, n_init=1)

In [ ]:
#kmeans.predict(residual[:,0:2].transpose())   # CUML
#D, I = kmeans.index.search(x, 1)             # FAISS

In [ ]:
# Set parameters
modelType = 1
loadBool = 1
modelPath = '/content/drive/MyDrive/NeuroResearch/Data/eegCompress/models/savedModel_03-28 19:04_-0.034.pt'

numSampleInput = 20
numSampleOutput = 1

if modelType == 0:
  inSize = nChannel * numSampleInput
  outSize = nChannel * numSampleOutput
  hiddenLayerSizeList = [300, 300, 300, 300, 300]
  layerSizeList = [inSize] + hiddenLayerSizeList + [outSize]
  model = model_0(nChannel, numSampleInput, layerSizeList)
  dataset = dataset_0(dataTensor, numSampleInput)
  sizeOfModel = modelSize(model)
elif modelType == 1:
  model = model_1(nChannel, numSampleInput)
  dataset = dataset_1(dataTensor, numSampleInput)
  sizeOfModel = modelSize(model)

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.0)


if loadBool:
  model, optimizer, totalEpoch, loss = loadModel(modelPath, model, optimizer, trainBool=True)
  print("Model has been loaded: " + modelPath)
else:
  totalEpoch = 0

print(model)

if torch.cuda.is_available():
  model.to('cuda')
  print("Model has been moved to GPU")
else:
  print("Model is on CPU")


model size: 0.119MB
Model has been loaded: /content/drive/MyDrive/NeuroResearch/Data/eegCompress/models/savedModel_03-28 19:04_-0.034.pt
model_1(
  (myNet): Sequential(
    (0): Conv1d(19, 50, kernel_size=(3,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(50, 50, kernel_size=(3,), stride=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(50, 50, kernel_size=(3,), stride=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=700, out_features=19, bias=True)
  )
)
Model has been moved to GPU


In [ ]:
residual.shape

(19,)

In [ ]:
kmeansCuml.predict(np.expand_dims(residual[i,:], 0))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
predicted = np.zeros_like(data)
predicted[:,0:numSampleInput] = data[:,0:numSampleInput]

for i in range(numSampleInput, nSample):
  if i % 100000 == 0:
    print(i)
  thisInput = torch.unsqueeze(torch.tensor(predicted[:,i-numSampleInput:i]),0)
  with torch.no_grad():
    if torch.cuda.is_available():
      thisInput = thisInput.to('cuda')
    thisOutput = model(thisInput).detach().cpu().numpy().squeeze()
    if np.any(np.isnan(thisOutput)):
      print("nan: " + str(i))
      break
  thisResidual = data[:,i] - thisOutput
  #_, index = kmeans.index.search(np.expand_dims(residual,0),1)
  #index = int(index[0][0])
  index = int(kmeansCuml.predict(np.expand_dims(thisResidual, 0))[0])

  predicted[:,i] = thisOutput + centroids[index,:]

nan: 9191


In [ ]:
path = '/content/drive/MyDrive/NeuroResearch/Data/eegCompress/processedData/origAndPredictedLossy.npz'
dataToSaveList = [data, predicted]
np.savez_compressed(path, *dataToSaveList)

# Scratch